James Gardner and S. Borhanian, 2022 
#### based on quick_start.ipynb by Borhanian 

want to analyse science case/s for CE only:

CE-N 40km with CE-S 40km or 20km

if done, then look at CE-S with one ET detector

In [26]:
from gwbench import network
from gwbench import wf_class as wfc
from gwbench import detector_response_derivatives as drd
from gwbench import injections

import os, sys
import numpy as np
# from p_tqdm import p_map
from p_tqdm import p_umap
from copy import deepcopy
import matplotlib.pyplot as plt

# https://stackoverflow.com/a/45669280; use as ``with HiddenPrints():''
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
def generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string,
                                locs, use_rot, output_path=None):
    """generate symbolic derivatives, from generate_lambdified_functions.py from S. Borhanian 2020
    use network's wf_model_name, wf_other_var_dic, deriv_symbs_string, and use_rot
    will print 'Done.' when finished unless all files already exist in which it will print as such
    
    # # how to print settings as a sanity check
    # print('wf_model_name = \'{}\''.format(wf.wf_model_name))
    # print('wf_other_var_dic = {}'.format(wf.wf_other_var_dic))
    # print('deriv_symbs_string = \'{}\''.format(deriv_symbs_string))
    # print('use_rot = %i'%use_rot)"""
    # skip if derivatives already exist
    file_names = ['par_deriv_WFM_'+wf_model_name+'_VAR_'+deriv_symbs_string.replace(' ', '_')+'_DET_'+key+'.dat' for key in locs]
    file_names.append('par_deriv_WFM_'+wf_model_name+'_VAR_'+deriv_symbs_string.replace(' ra', '').replace(' dec', '').replace(' psi', '').replace(' ', '_')+'_DET_'+'pl_cr'+'.dat')
    path = 'lambdified_functions/'
    file_names_existing = [file_name for file_name in file_names if os.path.isfile(path+file_name)]
    if len(file_names_existing) < len(file_names):
        # if a file doesn't exist, generate them all again
        # waveform
        wf = wfc.Waveform(wf_model_name, wf_other_var_dic)
        # lambidified detector reponses and derivatives
        drd.generate_det_responses_derivs_sym(wf, deriv_symbs_string, locs=locs, use_rot=use_rot,
                                              user_lambdified_functions_path=output_path)   
    else:
        print('All lambdified derivatives already exist.')

## Network initialisation

In [ ]:
# following quick_start.ipynb as a guide 

# initialisation is '<config>_<site>'
# other configurations (Tab.4): aLIGO,A+,V+,K+,Voyager-CBO,Voyager-PMO,ET 
# CE configs: <stage>-<arm length>-<optimisation> = CE1/CE2-10/20/30/40-CBO/PMO
# sites/locations (Tab.3): H,L,V,K,I,E,C,N,S; the latter three are Main, North, and South for CE

# choose the desired detectors
# CE-N 40km, CE-S 40km:
network_spec = ['CE1-40-CBO_C', 'CE1-40-CBO_S']
# network_spec = ['CE2-40-CBO_C', 'CE2-40-CBO_S'] # 2nd stage of development, compare as well
# # CE-N 40km, CE-S 20km:
# network_spec = ['CE1-40-CBO_C', 'CE1-20-PMO_S']
# network_spec = ['CE2-40-CBO_C', 'CE2-20-PMO_S'] # 2nd stage
locs = ['C', 'S']

# initialize the network with the desired detectors
net = network.Network(network_spec)

# frequency range
f = np.arange(5., 61.5, 2**-4) # i.e. looking for CBCs around 50 Hz, change for PMO

# choose the desired waveform 
wf_model_name = 'tf2' # TaylorF2, coded in gwbench, allows for symbolic differentiation
wf_other_var_dic = None # for tf2 or tf2_tidal
# other waveforms are available (e.g. tf2_tidal) but require different injection parameters

# pass the chosen waveform to the network for initialization
net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters
# for GW150914
inj_params = {
    'Mc':    30.9,
    'eta':   0.247,
    'chi1z': 0,
    'chi2z': 0,
    'DL':    475,
    'tc':    0,
    'phic':  0,
    'iota':  np.pi/4,
    'ra':    np.pi/4,
    'dec':   np.pi/4,
    'psi':   np.pi/4,
    'gmst0': 0}

# assign with respect to which parameters to take derivatives, for the FIM
deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'

# assign which parameters to convert to cos or log versions
conv_cos = ('iota', 'dec')
conv_log = ('Mc', 'DL')

# choose whether to take Earth's rotation into account
use_rot = 0
only_net = 1

# pass all these variables to the network
net.set_net_vars(
    f=f, inj_params=inj_params,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )
print('Network initialised')

## GW benchmarking

In [ ]:
# symbolic derivatives (faster)
generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, locs, use_rot)

# compute the WF polarizations and their derivatives
net.calc_wf_polarizations()
# --- numerical differentiation ---
# net.calc_wf_polarizations_derivs_num()
# --- symbolic differentiation ---
net.load_wf_polarizations_derivs_sym()
net.calc_wf_polarizations_derivs_sym()

# setup antenna patterns, location phase factors, and PSDs
net.setup_ant_pat_lpf_psds()
# results are accessed like this
# net.detectors[0].Fp # [i] for ith detector in network
# net.detectors[0].Fc # --> not frequency dependent?
# net.detectors[0].Flp
# net.detectors[0].psd # f in future calculations truncated to match psd

# compute the detector responses and their derivatives
# analogous to WF calculation
net.calc_det_responses()
# --- numerical differentiation ---
# net.calc_det_responses_derivs_num()
# --- symbolic differentiation ---
net.load_det_responses_derivs_sym()
net.calc_det_responses_derivs_sym()

# access results (either way) via
# net.detectors[0].hf
# net.detectors[0].del_hf

# calculate the network and detector SNRs
net.calc_snrs(only_net=only_net)
# print(net.snr, net.snr_sq, net.detectors[0].snr, net.detectors[0].snr_sq)

# calculate the network and detector Fisher matrices, condition numbers, ...
# ... covariance matrices, error estimates, and inversion errors
net.calc_errors(only_net=only_net) # finds FIMs, then inverts to find covariance matrix and error estimates of params
# print(net.fisher, net.cond_num, net.cov, net.errs, net.inv_err)
# print(net.detectors[0].fisher, net.detectors[0].cond_num, net.detectors[0].cov,
#       net.detectors[0].errs, net.detectors[0].inv_err)

# calculate the 90%-credible sky area (in [deg]^2)
net.calc_sky_area_90(only_net=only_net)
print('\nBenchmarking complete')

In [ ]:
# print the contents of the detector objects (inside the network)
# net.print_detectors()

# print the contents of the network objects
net.print_network()

In [ ]:
# net.get_snrs_errs_cov_fisher_inv_err_for_key(key='network')

In [ ]:
# # check if f truncated in any of the detectors to fit the psd
# for i in range(len(net.detectors)):
#     print(np.all(net.f == net.detectors[i].f))

### Replicating Fig 3 from Borhanian 2021 to test understanding, then try for CE only

In [27]:
# for GW170817 and tf2_tidal

# select network
# net, locs = network.Network(['aLIGO_H','aLIGO_L','aLIGO_V']), ['H', 'L', 'V']
net, locs = network.Network(['CE1-40-CBO_C', 'CE1-40-CBO_S']), ['C', 'S']
# to-do: for CE only, the FIM is ill-conditioned, how to fix this?

# not stated, using one from GW170817 paper
# start with 100 sample points, then move up from there (e.g. to 1e4)
fmin, fmax, fnum = 30, 2048, 100
f = np.linspace(fmin, fmax, fnum)

wf_model_name = 'tf2_tidal'
wf_other_var_dic = None
net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters for GW170817, reported median values (source-frame)
# using low-spin priors (Borhanian doesn't say which they used)
# https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.119.161101
# subsequent work (2019) has refined these values
base_measured_params = {
    'Mc':    1.188, # Msun
    'eta':   0.2485, # m1=1.48 Msun, m2=1.265 Msun, m2/m1=0.85, Mtot=2.74 Msun, eta=m1*m2/(m1+m2)**2
    'chi1z': 0,
    'chi2z': 0,
    'DL':    40, # MPc
    'tc':    0, # not quoted
    'phic':  0,
#     'iota':  np.pi/4, # 1000 random instances of these, measured: 55deg
#     'ra':    np.pi/4, # 1000 random instances of these
#     'dec':   np.pi/4, # 1000 random instances of these
    'psi':   np.pi/4, # not quoted, to-do: check effect
    'gmst0': 0, # not quoted but doesn't matter?
    'lam_t': 800, # combined dimensionless tidal deformability
    'delta_lam_t': 0 # not quoted, but approximating as zero because distributed around zero (check)
}

# assign with respect to which parameters to take derivatives, for the FIM, all 12 but not delta_lam_t (or gmst0)
deriv_symbs_string = 'Mc eta chi1z chi2z DL tc phic iota ra dec psi lam_t'

# assign which parameters to convert to log or cos versions for differentiation
conv_log = ('Mc', 'DL', 'lam_t')
conv_cos = ('iota', 'dec')

# choose whether to take Earth's rotation into account
use_rot = 0
# whether to calculate snr, errors, sky area for just the network and not the individual detectors
only_net = 1

# create lambdified derivatives for speed
generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, locs, use_rot)

# starting with just 10 instances, scale up to 1000 later
num_instances = 100 # 1000 is O(10 min) with numerical derivatives but is O(1 min) with symbolic derivatives
file_tag = f'{num_instances}instances_{net.label}'

All lambdified derivatives already exist.


In [28]:
# angles are sampled to avoid clumping at poles
iota_ra_dec_randoms = np.transpose(injections.angle_sampler(num_instances, np.random.randint(100))[:-1])

def calculate_snr_errs_skyarea(iota_ra_dec):
    """given an array of iota, ra, dec; return an array of the integrate snr, Mc and DL errors, and 90% sky area"""
    iota, ra, dec = iota_ra_dec
    inj_params = dict(**base_measured_params, iota=iota, ra=ra, dec=dec)

    # copy network to avoid parallel operations conflicting, is this an issue when Pool() makes separate instances?
    net_copy = deepcopy(net)

    net_copy.set_net_vars(
        f=f, inj_params=inj_params,
        deriv_symbs_string=deriv_symbs_string,
        conv_cos=conv_cos, conv_log=conv_log,
        use_rot=use_rot
    )

    with HiddenPrints():
        # compute the WF polarizations and their derivatives
        net_copy.calc_wf_polarizations()
        # --- numerical differentiation ---
        # net_copy.calc_wf_polarizations_derivs_num()
        # --- symbolic differentiation ---
        net_copy.load_wf_polarizations_derivs_sym()
        net_copy.calc_wf_polarizations_derivs_sym()

        # setup antenna patterns, location phase factors, and PSDs
        net_copy.setup_ant_pat_lpf_psds()

        # compute the detector responses and their derivatives
        net_copy.calc_det_responses()
        # --- numerical differentiation ---
        # net_copy.calc_det_responses_derivs_num()
        # --- symbolic differentiation ---
        net_copy.load_det_responses_derivs_sym()
        net_copy.calc_det_responses_derivs_sym()

        # calculate the network and detector SNRs
        net_copy.calc_snrs(only_net=only_net)
        # calculate the Fisher and covariance matrices, then error estimates
        net_copy.calc_errors(only_net=only_net) #cond_sup=# 1e15 (default) or None (allows all)
        # calculate the 90%-credible sky area (in [deg]^2)
        net_copy.calc_sky_area_90(only_net=only_net)

    if net_copy.wc_fisher: # i.e. net.cond_num < 1e15:
        # net_copy is automatically deleted once out of scope
        return net_copy.snr, net_copy.errs['log_Mc'], net_copy.errs['log_DL'], net_copy.errs['sky_area_90']
    else:
        # try again with different random values
        # to-do: add counter to quantify rate of occurance; seems to be v common with CE only, why?
        #print(f'{net_copy.cond_num:.3g} is ill-conditioned, try again')
        #return calculate_snr_errs_skyarea(np.transpose(injections.angle_sampler(1, np.random.randint(100))[:-1]))
        # alternative to guarantee halting 
        return np.full(4, np.nan)

# array to store integrated SNR, 1sigma error estimates (for Mc and DL), and 90% credible sky area
# must be careful with parallelising that the network is not used simultaneously
# keeping one cpu free to use laptop
results_snr_errs_skyarea = np.array(p_umap(calculate_snr_errs_skyarea, iota_ra_dec_randoms, num_cpus=3))

# save results
np.save(f'data_snr_errs_skyarea/results_snr_errs_skyarea_{file_tag}.npy', results_snr_errs_skyarea)

  0%|          | 0/100 [00:00<?, ?it/s]

In [29]:
# load results
results_snr_errs_skyarea = np.load(f'data_snr_errs_skyarea/results_snr_errs_skyarea_{file_tag}.npy')

# # filter out NaNs
# results_snr_errs_skyarea = results_snr_errs_skyarea[np.logical_not(np.isnan(results_snr_errs_skyarea))]

if len(results_snr_errs_skyarea[np.logical_not(np.isnan(results_snr_errs_skyarea))]) == 0:
    print('All values are NaN, FIM is ill-conditioned.')
else:
    print('Some values are not NaN.')

All values are NaN, FIM is ill-conditioned.


In [ ]:
# measured values
# given Appendix E: Asymmetric Systematic Uncertainties, estimating the st.dev. from asymm errs depends on your model
# in the Gaussian case, just average the upper and lower errors that correspond to 68% of the data
# given a 90% credibility interval (x-xlower,x+xupper), the st.dev. is approximated by 1/1.64*1/2(xlower+xupper)
meas_snr = 32.4
# too high eyeballing wrt Borhanian 2021?
meas_Mc_rel_err = 1/1.64*1/2*(0.004+0.002)/1.188 # 1.188+0.004-0.002, range for 90% credibility
# too low eyeballing wrt Borhanian 2021?
meas_LD_rel_err = 1/1.64*1/2*(8+14)/40 # 40+8−14
meas_sky_area = 28
meas_vals = meas_snr, meas_Mc_rel_err, meas_LD_rel_err, meas_sky_area

# plotting
# comparing symmetricised relative errors rather than 90% credible bounds
plt.rcParams.update({'font.size': 14})
fig, axs = plt.subplots(1, 4, figsize=(18, 2), sharey=True, gridspec_kw={'wspace':0.05, 'hspace':0})

for i, ax in enumerate(axs):
    data = results_snr_errs_skyarea[:,i]
    ax.hist(data, histtype='step', facecolor='b', label='tf2_tidal',
            bins=np.geomspace(data.min(), data.max(), 50))
    ax.axvline(meas_vals[i], color='grey')
    ax.set_xscale('log')
    # NB: when there's no major tick, the axis has no reference value
    ax.xaxis.set_minor_locator(plt.LogLocator(base=10.0, subs=0.1*np.arange(1, 10), numticks=10))
    ax.xaxis.set_minor_formatter(plt.NullFormatter())

axs[0].set_ylabel(r'GW170817')
axs[0].set_xlabel(r'integrated SNR, $\rho$')
# error in log(X) is the fractional error in X (i.e. (error in X)/X) by chain rule 
axs[1].set_xlabel(r'chirp mass, $\Delta\mathcal{M}/\mathcal{M}$')
axs[2].set_xlabel(r'luminosity distance, $\Delta D_L/D_L$')
axs[3].set_xlabel(r'sky area, $\Omega$ / $\mathrm{deg}^2$')
axs[0].legend()

fig.align_labels()
fig.savefig(f'GW170817_histograms_{file_tag}.pdf', bbox_inches='tight')
# plt.show(fig)
plt.close(fig)

# results for HLV disagree with Borhanian2021: sharpness of first twos' fall-offs, centre of snr distribution, measured Mc and DL, extent of sky area curve
# however, with a difference 1000 instances, the results change --- so maybe just a sampling issue

### Replicating Borhanian and Sathya 2022 injections and detection rates, then for CE only 

In [ ]:
# use injections.py to sample

In [37]:
# --- HLVKI+ ---
network_spec = ['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']
locs = list(set(x[-1] for x in network_spec))
net = network.Network(network_spec)

# network settings
use_rot = 1
conv_cos = ('dec', 'iota')
conv_log = ('Mc', 'DL', 'lam_t')

# injection settings
# number of injections per redshift bin (6 bins)
num_injs = 10 # start with 10, then build to 1e6
redshifted = 1

# --- BNS ---
wf_model_name = 'lal_bns'
wf_other_var_dic = dict(approximant='IMRPhenomD_NRTidalv2')
# parameters of sampling distributions from B&S 2022
mass_dict = dict(dist='gaussian', mean=1.35, sigma=0.15, mmin=1, mmax=2)
spin_dict = dict(geom='cartesian', dim=1, chi_lo=-0.05, chi_hi=0.05)
redshift_bins = ((0, 0.5, 7669), (0.5, 1, 3103), (1, 2, 4431), (2, 4, 5526), (4, 10, 7035), (10, 50, 2785))

for zmin, zmax, seed in redshift_bins:
    cosmo_dict = dict(sampler='uniform', zmin=zmin, zmax=zmax)
    # [Mc_vec, eta_vec, chi1x_vec, chi1y_vec, chi1z_vec, chi2x_vec, chi2y_vec, chi2z_vec, DL_vec, iota_vec, ra_vec, dec_vec, psi_vec, z_vec]
    inj_data = injections.injections_CBC_params_redshift(cosmo_dict, mass_dict, spin_dict, redshifted, num_injs=num_injs, seed=seed)
    print(inj_data)
    # to-do: pass injection data to benchmarking, then calculate a detection rate!
    
# numerical derivatives
step = 1e-9
method = 'central'
order = 2
d_order_n = 1

[array([1.71557549, 1.52548073, 1.39220165, 1.71002025, 1.52082262,
       1.37815347, 1.79770416, 1.4759012 , 1.51023084, 1.6240838 ]), array([0.24992066, 0.24997708, 0.2479219 , 0.2499399 , 0.24973456,
       0.24954208, 0.24913589, 0.24931697, 0.24996718, 0.2498733 ]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([-0.01996252, -0.00544674,  0.00757818, -0.01473179, -0.03383395,
       -0.02372966, -0.02410502,  0.0342316 , -0.00375813,  0.00318085]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([ 0.04268335, -0.00758601,  0.02768671,  0.01001657,  0.00506489,
       -0.01622505, -0.031138  , -0.00386361, -0.04221285,  0.04402079]), array([2480.49597308, 2749.52638283, 1147.04842716, 2012.56188358,
       1121.16291856,  589.92735875, 2469.64309462,  934.40871008,
        772.22511085, 1696.60169964]), array([1.29519379, 1.59257133, 1.2312829 , 1.6730409 , 1.72019956,

In [ ]:
inj_params = {
    'tc':    0, # 0 in Table 2
    'phic':  0, # 0 in Table 2
}